# Testing cosmogan
Aug 25, 2020

Borrowing pieces of code from : 

- https://github.com/pytorch/tutorials/blob/11569e0db3599ac214b03e01956c2971b02c64ce/beginner_source/dcgan_faces_tutorial.py
- https://github.com/exalearn/epiCorvid/tree/master/cGAN

In [16]:
import os
import random
import logging
import sys

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
#from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel
# import torch.fft

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import argparse
import time
from datetime import datetime
import glob
import pickle
import yaml
import collections
import socket
import shutil

# # Import modules from other files
# from utils import *
# from spec_loss import *

In [17]:
%matplotlib widget

## Modules

In [18]:
# Mod for 3D
def f_get_model(model_name,gdict):
    ''' Module to define Generator and Discriminator'''
    print("Model name",model_name)
    
    if model_name==2: #### Concatenate sigma input
        class Generator(nn.Module):
            def __init__(self, gdict):
                super(Generator, self).__init__()

                ## Define new variables from dict
                keys=['ngpu','nz','nc','ngf','kernel_size','stride','g_padding']
                ngpu, nz,nc,ngf,kernel_size,stride,g_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())

                self.main = nn.Sequential(
                    # nn.ConvTranspose3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                    nn.Linear(nz+1,nc*ngf*8**3),# 262144 
                    nn.BatchNorm3d(nc,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    View(shape=[-1,ngf*8,4,4,4]),
                    nn.ConvTranspose3d(ngf * 8, ngf * 4, kernel_size, stride, g_padding, output_padding=1, bias=False),
                    nn.BatchNorm3d(ngf*4,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    # state size. (ngf*4) x 8 x 8
                    nn.ConvTranspose3d( ngf * 4, ngf * 2, kernel_size, stride, g_padding, 1, bias=False),
                    nn.BatchNorm3d(ngf*2,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    # state size. (ngf*2) x 16 x 16
                    nn.ConvTranspose3d( ngf * 2, ngf, kernel_size, stride, g_padding, 1, bias=False),
                    nn.BatchNorm3d(ngf,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    # state size. (ngf) x 32 x 32
                    nn.ConvTranspose3d( ngf, nc, kernel_size, stride,g_padding, 1, bias=False),
                    nn.Tanh()
                )

            def forward(self, noise,labels):
                x=labels.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).float()
                gen_input=torch.cat((noise,x),-1)
                img=self.main(gen_input)

                return img

        class Discriminator(nn.Module):
            def __init__(self, gdict):
                super(Discriminator, self).__init__()

                ## Define new variables from dict
                keys=['ngpu','nz','nc','ndf','kernel_size','stride','d_padding']
                ngpu, nz,nc,ndf,kernel_size,stride,d_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())

                self.linear_transf=nn.Linear(4,4)
                self.main = nn.Sequential(
                    # input is (nc) x 64 x 64
                    # nn.Conv3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                    nn.Conv3d(nc+1, ndf,kernel_size, stride, d_padding,  bias=True),
                    nn.BatchNorm3d(ndf,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf) x 32 x 32
                    nn.Conv3d(ndf, ndf * 2, kernel_size, stride, d_padding, bias=True),
                    nn.BatchNorm3d(ndf * 2,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*2) x 16 x 16
                    nn.Conv3d(ndf * 2, ndf * 4, kernel_size, stride, d_padding, bias=True),
                    nn.BatchNorm3d(ndf * 4,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*4) x 8 x 8
                    nn.Conv3d(ndf * 4, ndf * 8, kernel_size, stride, d_padding, bias=True),
                    nn.BatchNorm3d(ndf * 8,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*8) x 4 x 4
                    nn.Flatten(),
                    nn.Linear(nc*ndf*8*8*8, 1)
        #             nn.Sigmoid()
                )

            def forward(self, img,labels):
                img_size=gdict['image_size']
                x=labels.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).repeat(1,1,1,1,4).float() # get to size (batch,1,1,1,4)
                x=self.linear_transf(x)
                x=torch.repeat_interleave(x,int((img_size*img_size*img_size)/4)) # get to size (batch* img^3)
                x=x.view(labels.size(0),1,img_size,img_size,img_size) ## Get to size (batch,1,img,img,img)
                
                ip=torch.cat((img,x),axis=1)
                
                results=[ip]
                lst_idx=[]
                for i,submodel in enumerate(self.main.children()):
                    mid_output=submodel(results[-1])
                    results.append(mid_output)
                    ## Select indices in list corresponding to output of Conv layers
                    if submodel.__class__.__name__.startswith('Conv'):
        #                 print(submodel.__class__.__name__)
        #                 print(mid_output.shape)
                        lst_idx.append(i)

                FMloss=True
                if FMloss:
                    ans=[results[1:][i] for i in lst_idx + [-1]]
                else :
                    ans=results[-1]
                
                return ans                
                

    elif model_name==3:#### Model 3: with ConditionalInstanceNorm2d,
        class ConditionalInstanceNorm2d(nn.Module):
            def __init__(self, num_features, num_params):
                super().__init__()
                self.num_features = num_features
                self.InstNorm = nn.InstanceNorm2d(num_features, affine=False)
                self.affine = nn.Linear(num_params, num_features * 2)
                self.affine.weight.data[:, :num_features].normal_(1, 0.02)  # Initialise scale at N(1, 0.02)
                self.affine.weight.data[:, num_features:].zero_()  # Initialise bias at 0

            def forward(self, x, y):
                out = self.InstNorm(x)
                gamma, beta = self.affine(y).chunk(2, 1)
                out = gamma.view(-1, self.num_features, 1, 1) * out + beta.view(-1, self.num_features, 1, 1)
                return out

        class ConditionalSequential(nn.Sequential):
            def __init__(self,*args):
                super(ConditionalSequential, self).__init__(*args)

            def forward(self, inputs, labels):
                for module in self:
                    if module.__class__ is ConditionalInstanceNorm2d:
                        inputs = module(inputs, labels.float())
                    else:
                        inputs = module(inputs)

                return inputs

        class Generator(nn.Module):
            def __init__(self, gdict):
                super(Generator, self).__init__()

                ## Define new variables from dict
                keys=['ngpu','nz','nc','ngf','kernel_size','stride','g_padding']
                ngpu, nz,nc,ngf,kernel_size,stride,g_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())

                self.main = ConditionalSequential(
                    # nn.ConvTranspose3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                    nn.Linear(nz,nc*ngf*8**3),# 262144
                    nn.BatchNorm3d(nc,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    View(shape=[-1,ngf*8,4,4,4]),
                    nn.ConvTranspose3d(ngf * 8, ngf * 4, kernel_size, stride, g_padding, output_padding=1, bias=False),
                    ConditionalInstanceNorm2d(ngf*4,1),
                    nn.ReLU(inplace=True),
                    # state size. (ngf*4) x 8 x 8
                    nn.ConvTranspose3d( ngf * 4, ngf * 2, kernel_size, stride, g_padding, 1, bias=False),
                    ConditionalInstanceNorm2d(ngf*2,1),
                    nn.ReLU(inplace=True),
                    # state size. (ngf*2) x 16 x 16
                    nn.ConvTranspose3d( ngf * 2, ngf, kernel_size, stride, g_padding, 1, bias=False),
                    ConditionalInstanceNorm2d(ngf,1),
                    nn.ReLU(inplace=True),
                    # state size. (ngf) x 32 x 32
                    nn.ConvTranspose3d( ngf, nc, kernel_size, stride,g_padding, 1, bias=False),
                    nn.Tanh()
                )
                
                
            def forward(self, noise,labels):
                img=self.main(noise,labels)

                return img

        class Discriminator(nn.Module):
            def __init__(self, gdict):
                super(Discriminator, self).__init__()

                ## Define new variables from dict
                keys=['ngpu','nz','nc','ndf','kernel_size','stride','d_padding']
                ngpu, nz,nc,ndf,kernel_size,stride,d_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())
                
                self.main = nn.Sequential(
                    # input is (nc) x 64 x 64
                    # nn.Conv3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                    nn.Conv3d(nc, ndf,kernel_size, stride, d_padding,  bias=True),
                    ConditionalInstanceNorm2d(ndf,1),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf) x 32 x 32
                    nn.Conv3d(ndf, ndf * 2, kernel_size, stride, d_padding, bias=True),
                    ConditionalInstanceNorm2d(ndf*2,1),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*2) x 16 x 16
                    nn.Conv3d(ndf * 2, ndf * 4, kernel_size, stride, d_padding, bias=True),
                    ConditionalInstanceNorm2d(ndf*4,1),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*4) x 8 x 8
                    nn.Conv3d(ndf * 4, ndf * 8, kernel_size, stride, d_padding, bias=True),
                    ConditionalInstanceNorm2d(ndf*8,1),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*8) x 4 x 4
                    nn.Flatten(),
                    nn.Linear(nc*ndf*8*8*8, 1)
        #             nn.Sigmoid()
                )
        
            def forward(self, ip,labels):   
                results=[ip]
                lst_idx=[]
                for i,submodel in enumerate(self.main.children()):
                    mid_output=submodel(results[-1])
                    results.append(mid_output)
                    ## Select indices in list corresponding to output of Conv layers
                    if submodel.__class__.__name__.startswith('Conv'):
        #                 print(submodel.__class__.__name__)
        #                 print(mid_output.shape)
                        lst_idx.append(i)

                FMloss=True
                if FMloss:
                    ans=[results[1:][i] for i in lst_idx + [-1]]
                else :
                    ans=results[-1]
                return ans

    return Generator, Discriminator

In [19]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
#     elif classname.find('Linear') != -1:
#         nn.init.normal_(m.weight.data, 1.0, 0.02)
        
# Generator Code
class View(nn.Module):
    def __init__(self, shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)

def f_gen_images(gdict,netG,optimizerG,sigma,ip_fname,op_loc,op_strg='inf_img_',op_size=500):
    '''Generate images for best saved models
     Arguments: gdict, netG, optimizerG, sigma (parameter value),
                 ip_fname: name of input file
                op_strg: [string name for output file]
                op_size: Number of images to generate
    '''

    nz,device=gdict['nz'],gdict['device']

    try:# handling cpu vs gpu
        if torch.cuda.is_available(): checkpoint=torch.load(ip_fname)
        else: checkpoint=torch.load(ip_fname,map_location=torch.device('cpu'))
    except Exception as e:
        print(e)
        print("skipping generation of images for ",ip_fname)
        return
    
    ## Load checkpoint
    if gdict['multi-gpu']:
        netG.module.load_state_dict(checkpoint['G_state'])
    else:
        netG.load_state_dict(checkpoint['G_state'])
    
    ## Load other stuff
    iters=checkpoint['iters']
    epoch=checkpoint['epoch']
    optimizerG.load_state_dict(checkpoint['optimizerG_state_dict'])
    
    # Generate batch of latent vectors
    noise = torch.randn(op_size, 1, 1, 1, nz, device=device) ## Mod for 3D
    tnsr_cosm_params=(torch.ones(op_size,device=device)*sigma).view(op_size,1)
    
    # Generate fake image batch with G
    netG.eval() ## This is required before running inference
    with torch.no_grad(): ## This is important. fails without it for multi-gpu
        gen = netG(noise,tnsr_cosm_params)
        gen_images=gen.detach().cpu().numpy()
        print(gen_images.shape)
    
    op_fname='%s_epoch-%s_step-%s.npy'%(op_strg,epoch,iters)
    np.save(op_loc+op_fname,gen_images)

    print("Image saved in ",op_fname)
    
def f_save_checkpoint(gdict,epoch,iters,best_chi1,best_chi2,netG,netD,optimizerG,optimizerD,save_loc):
    ''' Checkpoint model '''
    
    if gdict['multi-gpu']: ## Dataparallel
        torch.save({'epoch':epoch,'iters':iters,'best_chi1':best_chi1,'best_chi2':best_chi2,
                'G_state':netG.module.state_dict(),'D_state':netD.module.state_dict(),'optimizerG_state_dict':optimizerG.state_dict(),
                'optimizerD_state_dict':optimizerD.state_dict()}, save_loc) 
    else :
        torch.save({'epoch':epoch,'iters':iters,'best_chi1':best_chi1,'best_chi2':best_chi2,
                'G_state':netG.state_dict(),'D_state':netD.state_dict(),'optimizerG_state_dict':optimizerG.state_dict(),
                'optimizerD_state_dict':optimizerD.state_dict()}, save_loc)
    
def f_load_checkpoint(ip_fname,netG,netD,optimizerG,optimizerD,gdict):
    ''' Load saved checkpoint
    Also loads step, epoch, best_chi1, best_chi2'''
    
    try:
        checkpoint=torch.load(ip_fname)
    except Exception as e:
        print(e)
        print("skipping generation of images for ",ip_fname)
        raise SystemError
    
    ## Load checkpoint
    if gdict['multi-gpu']:
        netG.module.load_state_dict(checkpoint['G_state'])
        netD.module.load_state_dict(checkpoint['D_state'])
    else:
        netG.load_state_dict(checkpoint['G_state'])
        netD.load_state_dict(checkpoint['D_state'])
    
    optimizerD.load_state_dict(checkpoint['optimizerD_state_dict'])
    optimizerG.load_state_dict(checkpoint['optimizerG_state_dict'])
    
    iters=checkpoint['iters']
    epoch=checkpoint['epoch']
    best_chi1=checkpoint['best_chi1']
    best_chi2=checkpoint['best_chi2']

    netG.train()
    netD.train()
    
    return iters,epoch,best_chi1,best_chi2



In [20]:
####################
### Pytorch code ###
####################

## Mod for 3D

def f_get_rad(img):
    ''' Get the radial tensor for use in f_torch_get_azimuthalAverage '''
    
    height,width,depth=img.shape[-3:]
    # Create a grid of points with x and y and z coordinates
    z,y,x = np.indices([height,width,depth])
    
    center=[]
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0, (z.max()-z.min())/2.0])

    # Get the radial coordinate for every grid point. Array has the shape of image
    r= torch.tensor(np.sqrt((x-center[0])**2 + (y-center[1])**2 + (z-center[2])**2))
        
    # Get sorted radii
    ind = torch.argsort(torch.reshape(r, (-1,)))

    return r.detach(),ind.detach()

def f_torch_get_azimuthalAverage(image,r,ind):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 3D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    source: https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/
    """
    
#     height,width,depth=img.shape[-3:]
#     # Create a grid of points with x and y coordinates
#     z,y,x = np.indices([height,width,depth])
    
#     center=[]
#     if not center:
#         center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0, (z.max()-z.min())/2.0])

#     # Get the radial coordinate for every grid point. Array has the shape of image
#     r= torch.tensor(np.sqrt((x-center[0])**2 + (y-center[1])**2 + (z-center[2])**2))
        
#     # Get sorted radii
#     ind = torch.argsort(torch.reshape(r, (-1,)))

    r_sorted = torch.gather(torch.reshape(r, ( -1,)),0, ind)
    i_sorted = torch.gather(torch.reshape(image, ( -1,)),0, ind)
    
    # Get the integer part of the radii (bin size = 1)
    r_int=r_sorted.to(torch.int32)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = torch.reshape(torch.where(deltar)[0], (-1,))    # location of changes in radius
    nr = (rind[1:] - rind[:-1]).type(torch.float)       # number of radius bin

    # Cumulative sum to figure out sums for each radius bin
    
    csum = torch.cumsum(i_sorted, axis=-1)
    tbin = torch.gather(csum, 0, rind[1:]) - torch.gather(csum, 0, rind[:-1])
    radial_prof = tbin / nr

    return radial_prof

def f_torch_fftshift(real, imag):
    for dim in range(0, len(real.size())):
        real = torch.roll(real, dims=dim, shifts=real.size(dim)//2)
        imag = torch.roll(imag, dims=dim, shifts=imag.size(dim)//2)
    return real, imag

def f_torch_compute_spectrum(arr,r,ind):
    
    GLOBAL_MEAN=1.0
    arr=(arr-GLOBAL_MEAN)/(GLOBAL_MEAN)
    
    y1=torch.rfft(arr,signal_ndim=3,onesided=False)
    real,imag=f_torch_fftshift(y1[:,:,:,0],y1[:,:,:,1])    ## last index is real/imag part  ## Mod for 3D
    
#     # For pytorch 1.8
#     y1=torch.fft.fftn(arr,dim=(-3,-2,-1))
#     real,imag=f_torch_fftshift(y1.real,y1.imag)    
    
    y2=real**2+imag**2     ## Absolute value of each complex number
    z1=f_torch_get_azimuthalAverage(y2,r,ind)     ## Compute radial profile
    return z1


def f_torch_compute_batch_spectrum(arr,r,ind):
    
    batch_pk=torch.stack([f_torch_compute_spectrum(i,r,ind) for i in arr])
    
    return batch_pk

def f_torch_image_spectrum(x,num_channels,r,ind):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    mean=[[] for i in range(num_channels)]    
    sdev=[[] for i in range(num_channels)]    
    
    for i in range(num_channels):
        arr=x[:,i,:,:,:] # Mod for 3D
        batch_pk=f_torch_compute_batch_spectrum(arr,r,ind)
        mean[i]=torch.mean(batch_pk,axis=0)
#         sdev[i]=torch.std(batch_pk,axis=0)/np.sqrt(batch_pk.shape[0])
#         sdev[i]=torch.std(batch_pk,axis=0)
        sdev[i]=torch.var(batch_pk,axis=0)
    
    mean=torch.stack(mean)
    sdev=torch.stack(sdev)
        
    return mean,sdev

def f_compute_hist(data,bins):
    
    try: 
        hist_data=torch.histc(data,bins=bins)
        ## A kind of normalization of histograms: divide by total sum
        hist_data=(hist_data*bins)/torch.sum(hist_data)
    except Exception as e:
        print(e)
        hist_data=torch.zeros(bins)

    return hist_data

### Losses 
def loss_spectrum(spec_mean,spec_mean_ref,spec_std,spec_std_ref,image_size,lambda_spec_mean,lambda_spec_var):
    ''' Loss function for the spectrum : mean + variance 
    Log(sum( batch value - expect value) ^ 2 )) '''
    
    idx=int(image_size/2) ### For the spectrum, use only N/2 indices for loss calc.
    ### Warning: the first index is the channel number.For multiple channels, you are averaging over them, which is fine.
        
    spec_mean=torch.log(torch.mean(torch.pow(spec_mean[:,:idx]-spec_mean_ref[:,:idx],2)))
    spec_sdev=torch.log(torch.mean(torch.pow(spec_std[:,:idx]-spec_std_ref[:,:idx],2)))
    
    lambda1=lambda_spec_mean;
    lambda2=lambda_spec_var;
    ans=lambda1*spec_mean+lambda2*spec_sdev
    
    if torch.isnan(spec_sdev).any():    print("spec loss with nan",ans)
    
    return ans
    
def loss_hist(hist_sample,hist_ref):
    
    lambda1=1.0
    return lambda1*torch.log(torch.mean(torch.pow(hist_sample-hist_ref,2)))

def f_FM_loss(real_output,fake_output,lambda_fm,gdict):
    '''
    Module to implement Feature-Matching loss. Reads all but last elements of Discriminator ouput
    '''
    FM=torch.Tensor([0.0]).to(gdict['device'])
    for i,j in zip(real_output[:-1],fake_output[:-1]):
        real_mean=torch.mean(i)
        fake_mean=torch.mean(j)
        FM=FM.clone()+torch.sum(torch.square(real_mean-fake_mean))
    return lambda_fm*FM

def f_gp_loss(grads,l=1.0):
    '''
    Module to implement gradient penalty loss.
    '''
    loss=torch.mean(torch.sum(torch.square(grads),dim=[1,2,3]))
    return l*loss

def f_get_loss_cond(loss_type,img_tensor,cosm_params,gdict,bins=None,hist_val_tnsr=None,spec_mean_tnsr=None,spec_sdev_tnsr=None,r=None,ind=None,real_output=None,fake_output=None,grads=None):
    ''' Module to compute one of the losses for conditional GAN '''
    
    loss_tensor=torch.zeros(len(gdict['sigma_list']),device=gdict['device'])
    
    for count,i in enumerate(gdict['sigma_list']):   
        idxs=torch.where(cosm_params==i)[0] ## Get indices for that category
        if idxs.size(0)>1: 
            num_frac=idxs.size(0)/img_tensor.shape[0] ## Fraction of points in the category
            img=img_tensor[idxs]
            if loss_type=='hist':
                loss_tensor[count]=loss_hist(f_compute_hist(img,bins),hist_val_tnsr[count])*num_frac
            elif loss_type=='spec':
                mean,sdev=f_torch_image_spectrum(f_invtransform(img),1,r,ind)
                loss_tensor[count]=loss_spectrum(mean,spec_mean_tnsr[count],sdev,spec_sdev_tnsr[count],gdict['image_size'],gdict['lambda_spec_mean'],gdict['lambda_spec_var'])*num_frac
            elif loss_type=='fm':
                loss_tensor[count]=f_FM_loss(real_output,fake_output,gdict['lambda_fm'],gdict)
            elif loss_type=='gp':
                loss_tensor[count]=f_gp_loss(grads,gdict['lambda_gp'])

    loss=loss_tensor.sum()
            
    return loss



## Train loop

In [21]:
### Train code ###
def f_train_loop(dataloader,metrics_df,gdict,fixed_noise,mean_spec_val,sdev_spec_val,hist_val,r,ind):
    ''' Train single epoch '''

    ## Define new variables from dict
    keys=['image_size','start_epoch','epochs','iters','best_chi1','best_chi2','save_dir','device','flip_prob','nz','batch_size','bns']
    image_size,start_epoch,epochs,iters,best_chi1,best_chi2,save_dir,device,flip_prob,nz,batchsize,bns=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())
    
    for epoch in range(start_epoch,epochs):
        t_epoch_start=time.time()
        for count, data in enumerate(dataloader):

            ####### Train GAN ########
            netG.train(); netD.train();  ### Need to add these after inference and before training

            tme1=time.time()
            ### Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            netD.zero_grad()

            real_cpu = data[0].to(device)
            real_cosm_params=data[1].to(device)
            real_cpu.requires_grad=True
            
            b_size = real_cpu.size(0)
            real_label = torch.full((b_size,), 1, device=device,dtype=float)
            fake_label = torch.full((b_size,), 0, device=device,dtype=float)
            g_label = torch.full((b_size,), 1, device=device,dtype=float) ## No flipping for Generator labels
            # Flip labels with probability flip_prob
            for idx in np.random.choice(np.arange(b_size),size=int(np.ceil(b_size*flip_prob))):
                real_label[idx]=0; fake_label[idx]=1

            # Generate fake image batch with G
            noise = torch.randn(b_size, 1, 1, 1, nz, device=device) ### Mod for 3D
            rnd_idx=torch.randint(len(gdict['sigma_list']),(gdict['batch_size'],1),device=gdict['device'])
            fake_cosm_params=torch.tensor([gdict['sigma_list'][i] for i in rnd_idx.long()],device=gdict['device']).unsqueeze(-1)

            fake = netG(noise,fake_cosm_params)         

            # Forward pass real batch through D
            real_output = netD(real_cpu,real_cosm_params)
            errD_real = criterion(real_output[-1].view(-1), real_label.float())
            errD_real.backward(retain_graph=True)
            D_x = real_output[-1].mean().item()

            # Forward pass fake batch through D
            fake_output = netD(fake.detach(),fake_cosm_params)  # The detach is important
            errD_fake = criterion(fake_output[-1].view(-1), fake_label.float())
            errD_fake.backward(retain_graph=True)
            D_G_z1 = fake_output[-1].mean().item()
            
            errD = errD_real + errD_fake 

            if gdict['lambda_gp']: ## Add gradient - penalty loss
#                 f_get_loss_cond(img_tensor,cosm_params,gdict,bins=None,hist_val_tnsr=None,spec_mean_tnsr=None,spec_sdev_tnsr=None,r=None,ind=None,real_output=None,fake_output=None,grads=None)
                
                grads=torch.autograd.grad(outputs=real_output[-1],inputs=real_cpu,grad_outputs=torch.ones_like(real_output[-1]),allow_unused=False,create_graph=True)[0]
                gp_loss=f_get_loss_cond('gp',fake,fake_cosm_params,gdict,grads=grads)
                gp_loss.backward(retain_graph=True)
                errD = errD + gp_loss
            else:
                gp_loss=torch.Tensor([np.nan])

            optimizerD.step()

            ###Update G network: maximize log(D(G(z)))
            netG.zero_grad()
            output = netD(fake,fake_cosm_params)
            errG_adv = criterion(output[-1].view(-1), g_label.float())
            # Histogram pixel intensity loss
            hist_loss=f_get_loss_cond('hist',fake,fake_cosm_params,gdict,bins=gdict['bns'],hist_val_tnsr=hist_val)

            # Add spectral loss
            mean,sdev=f_torch_image_spectrum(f_invtransform(fake),1,r.to(device),ind.to(device))
            spec_loss=f_get_loss_cond('spec',fake,fake_cosm_params,gdict,spec_mean_tnsr=mean_spec_val,spec_sdev_tnsr=sdev_spec_val,r=r,ind=ind)
            
            errG=errG_adv
            if gdict['lambda_spec_mean']: errG = errG+ spec_loss 
            if gdict['lambda_fm']:## Add feature matching loss
                fm_loss=f_get_loss_cond('fm',fake,fake_cosm_params,gdict,real_output=[i.detach() for i in real_output],fake_output=output)
                errG= errG+ fm_loss
            else: 
                fm_loss=torch.Tensor([np.nan])

            if torch.isnan(errG).any():
                logging.info(errG)
                raise SystemError
            
            # Calculate gradients for G
            errG.backward()
            D_G_z2 = output[-1].mean().item()
            
            ### Implement Gradient clipping
            if gdict['grad_clip']:
                nn.utils.clip_grad_norm_(netG.parameters(),gdict['grad_clip'])
                nn.utils.clip_grad_norm_(netD.parameters(),gdict['grad_clip'])

#             optimizerD.step()
            optimizerG.step()

            tme2=time.time()
            ####### Store metrics ########
            # Output training stats
            if gdict['world_rank']==0:
                if ((count % gdict['checkpoint_size'] == 0)):
                    logging.info('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_adv: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                          % (epoch, epochs, count, len(dataloader), errD.item(), errG_adv.item(),errG.item(), D_x, D_G_z1, D_G_z2)),
                    logging.info("Spec loss: %s,\t hist loss: %s"%(spec_loss.item(),hist_loss.item())),
                    logging.info("Training time for step %s : %s"%(iters, tme2-tme1))

                # Save metrics
                cols=['step','epoch','Dreal','Dfake','Dfull','G_adv','G_full','spec_loss','hist_loss','fm_loss','gp_loss','D(x)','D_G_z1','D_G_z2','time']
                vals=[iters,epoch,errD_real.item(),errD_fake.item(),errD.item(),errG_adv.item(),errG.item(),spec_loss.item(),hist_loss.item(),fm_loss.item(),gp_loss.item(),D_x,D_G_z1,D_G_z2,tme2-tme1]
                for col,val in zip(cols,vals):  metrics_df.loc[iters,col]=val

                ### Checkpoint the best model
                checkpoint=True
                iters += 1  ### Model has been updated, so update iters before saving metrics and model.

                ### Compute validation metrics for updated model
                netG.eval()
                with torch.no_grad():
                    fake = netG(fixed_noise,fixed_cosm_params)
                    hist_chi=f_get_loss_cond('hist',fake,fixed_cosm_params,gdict,bins=gdict['bns'],hist_val_tnsr=hist_val)
                    spec_chi=f_get_loss_cond('spec',fake,fixed_cosm_params,gdict,spec_mean_tnsr=mean_spec_val,spec_sdev_tnsr=sdev_spec_val,r=r,ind=ind)

                # Storing chi for next step
                for col,val in zip(['spec_chi','hist_chi'],[spec_chi.item(),hist_chi.item()]):  metrics_df.loc[iters,col]=val            

                # Checkpoint model for continuing run
                if count == len(dataloader)-1: ## Check point at last step of epoch
                    f_save_checkpoint(gdict,epoch,iters,best_chi1,best_chi2,netG,netD,optimizerG,optimizerD,save_loc=save_dir+'/models/checkpoint_last.tar')  

                if (checkpoint and (epoch > 1)): # Choose best models by metric
                    if hist_chi< best_chi1:
                        f_save_checkpoint(gdict,epoch,iters,best_chi1,best_chi2,netG,netD,optimizerG,optimizerD,save_loc=save_dir+'/models/checkpoint_best_hist.tar')
                        best_chi1=hist_chi.item()
                        logging.info("Saving best hist model at epoch %s, step %s."%(epoch,iters))

                    if  spec_chi< best_chi2:
                        f_save_checkpoint(gdict,epoch,iters,best_chi1,best_chi2,netG,netD,optimizerG,optimizerD,save_loc=save_dir+'/models/checkpoint_best_spec.tar')
                        best_chi2=spec_chi.item()
                        logging.info("Saving best spec model at epoch %s, step %s"%(epoch,iters))

                    if iters in gdict['save_steps_list']:
                        f_save_checkpoint(gdict,epoch,iters,best_chi1,best_chi2,netG,netD,optimizerG,optimizerD,save_loc=save_dir+'/models/checkpoint_{0}.tar'.format(iters))
                        logging.info("Saving given-step at epoch %s, step %s."%(epoch,iters))

                # Save G's output on fixed_noise
                if ((iters % gdict['checkpoint_size'] == 0) or ((epoch == epochs-1) and (count == len(dataloader)-1))):
                    netG.eval()
                    with torch.no_grad():
                        for c_pars in gdict['sigma_list']:
                            tnsr_cosm_params=(torch.ones(batchsize,device=device)*c_pars).view(batchsize,1)
                            fake = netG(fixed_noise,tnsr_cosm_params).detach().cpu()
                            img_arr=np.array(fake)
                            fname='gen_img_label-%s_epoch-%s_step-%s'%(c_pars,epoch,iters)
                            np.save(save_dir+'/images/'+fname,img_arr)
        
        t_epoch_end=time.time()
        if gdict['world_rank']==0:
            logging.info("Time taken for epoch %s, count %s: %s for rank %s"%(epoch,count,t_epoch_end-t_epoch_start,gdict['world_rank']))
            # Save Metrics to file after each epoch
            metrics_df.to_pickle(save_dir+'/df_metrics.pkle')
            logging.info("best chis: {0}, {1}".format(best_chi1,best_chi2))



## Start

In [22]:

########## Modules
### Setup modules ###
def f_manual_add_argparse():
    ''' use only in jpt notebook'''
    args=argparse.Namespace()
    args.config='config_3d_cgan.yaml'
    args.mode='fresh'
    args.ip_fldr=''
    args.local_rank=0
    args.facility='cori'
    args.distributed=False

#     args.mode='continue'
#     args.ip_fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201211_093818_nb_test/'
    
    return args

def f_parse_args():
    """Parse command line arguments.Only for .py file"""
    parser = argparse.ArgumentParser(description="Run script to train GAN using pytorch", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    add_arg = parser.add_argument
    
    add_arg('--config','-cfile',  type=str, default='config_2d_Cgan.yaml', help='Whether to start fresh run or continue previous run')
    add_arg('--mode','-m',  type=str, choices=['fresh','continue'],default='fresh', help='Whether to start fresh run or continue previous run')
    add_arg('--ip_fldr','-ip',  type=str, default='', help='The input folder for resuming a checkpointed run')
    add_arg("--local_rank", default=0, type=int,help='Local rank of GPU on node. Using for pytorch DDP. ')
    add_arg("--facility", default='cori', choices=['cori','summit'],type=str,help='Facility: cori or summit ')
    add_arg("--ddp", dest='distributed' ,default=False,action='store_true',help='use Distributed DataParallel for Pytorch or DataParallel')
    
    return parser.parse_args()


def try_barrier(rank):
    """
    Used in Distributed data parallel
    Attempt a barrier but ignore any exceptions
    """
    print('BAR %d'%rank)
    try:
        dist.barrier()
    except:
        pass

def f_init_gdict(args,gdict):
    ''' Create global dictionary gdict from args and config file'''
    
    ## read config file
    config_file=args.config
    with open(config_file) as f:
        config_dict= yaml.load(f, Loader=yaml.SafeLoader)
        
    gdict=config_dict['parameters']

    args_dict=vars(args)
    ## Add args variables to gdict
    for key in args_dict.keys():
        gdict[key]=args_dict[key]

    if gdict['distributed']: 
        assert not gdict['lambda_gp'],"GP couplings is %s. Cannot use Gradient penalty loss in pytorch DDP"%(gdict['lambda_gp'])
    else : print("Not using DDP")
    return gdict


def f_sample_data(ip_tensor,params_tensor,rank=0,num_ranks=1):
    '''
    Module to load part of dataset depending on world_rank.
    '''
    data_size=ip_tensor.shape[0]
    size=data_size//num_ranks
    print("Using data indices %s-%s for rank %s"%(rank*(size),(rank+1)*size,rank))
    dataset=TensorDataset(ip_tensor[rank*(size):(rank+1)*size],params_tensor[rank*(size):(rank+1)*size])
    ### 
    if gdict['batch_size']>size:
        print("Caution: batchsize %s is less than samples per GPU."%(gdict['batch_size'],size))
        raise SystemExit
    
    data_loader=DataLoader(dataset,batch_size=gdict['batch_size'],shuffle=True,num_workers=0,drop_last=True)
    return data_loader


def f_load_data_precompute(gdict):
    #################################
    ####### Read data and precompute ######
    ### Read input data from different files
    for count,sigma in enumerate(gdict['sigma_list']):
        fname=gdict['ip_fname']+'/norm_1_sig_%s_train_val.npy'%(sigma)
        x=np.load(fname,mmap_mode='r')[:gdict['num_imgs']].transpose(0,1,2,3,4).copy() ## Mod for 3D
        size=x.shape[0]
        y=sigma*np.ones(size)

        if count==0:
            img=x[:]
            c_pars=y[:]
        else: 
            img=np.vstack([img,x]) # Store images
            c_pars=np.hstack([c_pars,y]) # Store cosmological parameters

    ### Manually shuffling numpy arrays to mix sigma values
    size=img.shape[0]
    idxs=np.random.choice(size,size=size,replace=False)
    img=img[idxs]
    c_pars=c_pars[idxs]
    ## convert to tensors
    t_img=torch.from_numpy(img)
    cosm_params=torch.Tensor(c_pars).view(size,1)
    logging.info("%s, %s"%(cosm_params.shape,t_img.shape))
    
#     dataset=TensorDataset(t_img,cosm_params)
#     dataloader=DataLoader(dataset,batch_size=gdict['batch_size'],shuffle=True,num_workers=0,drop_last=True)
    data_loader=f_sample_data(t_img,cosm_params,gdict['world_rank'],gdict['world_size'])
    print("Size of dataset for GPU %s : %s"%(gdict['world_rank'],len(data_loader.dataset)))
    
    # Precompute metrics with validation data for computing losses
    with torch.no_grad():
        spec_mean_list=[];spec_sdev_list=[];hist_val_list=[]
        
        for count,sigma in enumerate(gdict['sigma_list']):
            ip_fname=gdict['ip_fname']+'/norm_1_sig_%s_train_val.npy'%(sigma)
            val_img=np.load(ip_fname,mmap_mode='r')[-100:].transpose(0,1,2,3,4).copy() ## Mod for 3D
            t_val_img=torch.from_numpy(val_img).to(gdict['device'])

            # Precompute radial coordinates
            if count==0: 
                r,ind=f_get_rad(img)
                r=r.to(gdict['device']); ind=ind.to(gdict['device'])
            # Stored mean and std of spectrum for full input data once
            mean_spec_val,sdev_spec_val=f_torch_image_spectrum(f_invtransform(t_val_img),1,r,ind)
            hist_val=f_compute_hist(t_val_img,bins=gdict['bns'])
            
            spec_mean_list.append(mean_spec_val)
            spec_sdev_list.append(sdev_spec_val)
            hist_val_list.append(hist_val)
        spec_mean_tnsr=torch.stack(spec_mean_list)
        spec_sdev_tnsr=torch.stack(spec_sdev_list)
        hist_val_tnsr=torch.stack(hist_val_list)
        
        del val_img; del t_val_img; del img; del t_img; del spec_mean_list; del spec_sdev_list; del hist_val_list        

    return data_loader,spec_mean_tnsr,spec_sdev_tnsr,hist_val_tnsr,r,ind


def f_init_GAN(gdict,print_model=False):
    # Define Models
    logging.info("Building GAN networks")
    
    ## Choose model
    Generator, Discriminator=f_get_model(gdict['model'],gdict)

    # Create Generator
    netG = Generator(gdict).to(gdict['device'])
    netG.apply(weights_init)
    # Create Discriminator
    netD = Discriminator(gdict).to(gdict['device'])
    netD.apply(weights_init)
    
    if print_model:
        if gdict['world_rank']==0:
            print(netG)
        #     summary(netG,(1,1,64))
            print(netD)
        #     summary(netD,(1,128,128))
            print("Number of GPUs used %s"%(gdict['ngpu']))

    if (gdict['multi-gpu']):
        if not gdict['distributed']:
            netG = nn.DataParallel(netG, list(range(gdict['ngpu'])))
            netD = nn.DataParallel(netD, list(range(gdict['ngpu'])))
        else:
            netG=DistributedDataParallel(netG,device_ids=[gdict['local_rank']],output_device=[gdict['local_rank']])
            netD=DistributedDataParallel(netD,device_ids=[gdict['local_rank']],output_device=[gdict['local_rank']])
    
    #### Initialize networks ####
    # criterion = nn.BCELoss()
    criterion = nn.BCEWithLogitsLoss()

    if gdict['mode']=='fresh':
        optimizerD = optim.Adam(netD.parameters(), lr=gdict['learn_rate'], betas=(gdict['beta1'], 0.999),eps=1e-7)
        optimizerG = optim.Adam(netG.parameters(), lr=gdict['learn_rate'], betas=(gdict['beta1'], 0.999),eps=1e-7)
        ### Initialize variables      
        iters,start_epoch,best_chi1,best_chi2=0,0,1e10,1e10    

    ### Load network weights for continuing run
    elif gdict['mode']=='continue':
        iters,start_epoch,best_chi1,best_chi2=f_load_checkpoint(gdict['save_dir']+'/models/checkpoint_last.tar',netG,netD,optimizerG,optimizerD,gdict) 
        logging.info("Continuing existing run. Loading checkpoint with epoch {0} and step {1}".format(start_epoch,iters))
        start_epoch+=1  ## Start with the next epoch  

    ## Add to gdict
    for key,val in zip(['best_chi1','best_chi2','iters','start_epoch'],[best_chi1,best_chi2,iters,start_epoch]): gdict[key]=val

    return netG,netD,criterion,optimizerD,optimizerG


def f_setup(gdict,log):
    ''' 
    Set up directories, Initialize random seeds, add GPU info, add logging info.
    '''
    
    torch.backends.cudnn.benchmark=True
#     torch.autograd.set_detect_anomaly(True)

    ## New additions. Code taken from Jan B.
    os.environ['MASTER_PORT'] = "8885"

    if gdict['facility']=='summit':
        get_master = "echo $(cat {} | sort | uniq | grep -v batch | grep -v login | head -1)".format(os.environ['LSB_DJOB_HOSTFILE'])
        os.environ['MASTER_ADDR'] = str(subprocess.check_output(get_master, shell=True))[2:-3]
        os.environ['WORLD_SIZE'] = os.environ['OMPI_COMM_WORLD_SIZE']
        os.environ['RANK'] = os.environ['OMPI_COMM_WORLD_RANK']
        gdict['local_rank'] = int(os.environ['OMPI_COMM_WORLD_LOCAL_RANK'])
    else:
        if gdict['distributed']:
            os.environ['WORLD_SIZE'] = os.environ['SLURM_NTASKS']
            os.environ['RANK'] = os.environ['SLURM_PROCID']
            gdict['local_rank'] = int(os.environ['SLURM_LOCALID'])

    ## Special declarations
    gdict['ngpu']=torch.cuda.device_count()
    gdict['device']=torch.device("cuda" if (torch.cuda.is_available()) else "cpu")
    gdict['multi-gpu']=True if (gdict['device'].type == 'cuda') and (gdict['ngpu'] > 1) else False 
    
    ########################
    ###### Set up Distributed Data parallel ######
    if gdict['distributed']:
#         gdict['local_rank']=args.local_rank  ## This is needed when using pytorch -m torch.distributed.launch
        gdict['world_size']=int(os.environ['WORLD_SIZE'])
        torch.cuda.set_device(gdict['local_rank']) ## Very important
        dist.init_process_group(backend='nccl', init_method="env://")  
        gdict['world_rank']= dist.get_rank()
        
        print("World size %s, world rank %s, local rank %s, hostname %s, GPUs on node %s\n"%(gdict['world_size'],gdict['world_rank'],gdict['local_rank'],socket.gethostname(),gdict['ngpu']))
        device = torch.cuda.current_device()
        
        # Divide batch size by number of GPUs
#         gdict['batch_size']=gdict['batch_size']//gdict['world_size']
    else:
        gdict['world_size'],gdict['world_rank'],gdict['local_rank']=1,0,0
    
    ########################
    ###### Set up directories #######
    ### sync up so that time is the same for each GPU for DDP
    if gdict['mode']=='fresh':
        ### Create prefix for foldername            
        if gdict['world_rank']==0: ### For rank=0, create directory name string and make directories
            dt_strg=datetime.now().strftime('%Y%m%d_%H%M%S') ## time format
            dt_lst=[int(i) for i in dt_strg.split('_')] # List storing day and time            
            dt_tnsr=torch.LongTensor(dt_lst).to(gdict['device'])  ## Create list to pass to other GPUs 

        else: dt_tnsr=torch.Tensor([0,0]).long().to(gdict['device'])
        ### Pass directory name to other ranks
        if gdict['distributed']: dist.broadcast(dt_tnsr, src=0)

        gdict['save_dir']=gdict['op_loc']+str(int(dt_tnsr[0]))+'_'+str(int(dt_tnsr[1]))+'_'+gdict['run_suffix']
        
        if gdict['world_rank']==0: # Create directories for rank 0
            ### Create directories
            if not os.path.exists(gdict['save_dir']):
                os.makedirs(gdict['save_dir']+'/models')
                os.makedirs(gdict['save_dir']+'/images')
                shutil.copy(gdict['config'],gdict['save_dir'])            
    elif gdict['mode']=='continue': ## For checkpointed runs
        gdict['save_dir']=args.ip_fldr
        ### Read loss data
        with open (gdict['save_dir']+'df_metrics.pkle','rb') as f:
            metrics_dict=pickle.load(f)
   
    ########################
    ### Initialize random seed
    
    manualSeed = np.random.randint(1, 10000) if gdict['seed']=='random' else int(gdict['seed'])
#     print("Seed",manualSeed,gdict['world_rank'])
    random.seed(manualSeed)
    np.random.seed(manualSeed)
    torch.manual_seed(manualSeed)
    torch.cuda.manual_seed_all(manualSeed)
    
    if gdict['deterministic']:
        logging.info("Running with deterministic sequence. Performance will be slower")
        torch.backends.cudnn.deterministic=True
#         torch.backends.cudnn.enabled = False
        torch.backends.cudnn.benchmark = False        
    
    ########################
    if log:
        ### Write all logging.info statements to stdout and log file
        logfile=gdict['save_dir']+'/log.log'
        if gdict['world_rank']==0:
            logging.basicConfig(level=logging.DEBUG, filename=logfile, filemode="a+", format="%(asctime)-15s %(levelname)-8s %(message)s")

            Lg = logging.getLogger()
            Lg.setLevel(logging.DEBUG)
            lg_handler_file = logging.FileHandler(logfile)
            lg_handler_stdout = logging.StreamHandler(sys.stdout)
            Lg.addHandler(lg_handler_file)
            Lg.addHandler(lg_handler_stdout)

            logging.info('Args: {0}'.format(args))
            logging.info('Start: %s'%(datetime.now().strftime('%Y-%m-%d  %H:%M:%S')))
        
        if gdict['distributed']:  try_barrier(gdict['world_rank'])

        if gdict['world_rank']!=0:
                logging.basicConfig(level=logging.DEBUG, filename=logfile, filemode="a+", format="%(asctime)-15s %(levelname)-8s %(message)s")


In [23]:
def f_sample_data(ip_arr,params_arr,rank=0,num_ranks=1):
    '''
    Module to load part of dataset depending on world_rank.
    '''
    data_size=ip_arr.shape[0]
    size=data_size//num_ranks
    
    ### Get a set of random indices from numpy array
    idxs=np.arange(ip_arr.shape[0])
    np.random.shuffle(idxs)
    rnd_idxs=idxs[rank*(size):(rank+1)*size]
    
    t_img=torch.from_numpy(ip_arr[rnd_idxs].copy())
    cosm_params=torch.Tensor(params_arr[rnd_idxs]).view(size,1)
    logging.info("%s, %s"%(cosm_params.shape,t_img.shape))

    print(t_img.shape)
    dataset=TensorDataset(t_img,cosm_params)
    del t_img
    ### 
    if gdict['batch_size']>size:
        print("Caution: batchsize %s is less than samples per GPU."%(gdict['batch_size'],size))
        raise SystemExit
    
    data_loader=DataLoader(dataset,batch_size=gdict['batch_size'],shuffle=True,num_workers=0,drop_last=True)
    return data_loader


def f_load_data_precompute(gdict):
    #################################
    ####### Read data and precompute ######
    ### Read input data from different files
    for count,sigma in enumerate(gdict['sigma_list']):
        fname=gdict['ip_fname']+'/norm_1_sig_%s_train_val.npy'%(sigma)
        x=np.load(fname,mmap_mode='r')[:gdict['num_imgs']].transpose(0,1,2,3,4) ## Mod for 3D
        size=x.shape[0]
        y=sigma*np.ones(size)
        
        if count==0:
            img=x[:]
            c_pars=y[:]
        else: 
            img=np.vstack([img,x]) # Store images
            c_pars=np.hstack([c_pars,y]) # Store cosmological parameters
    
    ### Manually shuffling numpy arrays to mix sigma values
    size=img.shape[0]
    idxs=np.random.choice(size,size=size,replace=False)
    img=img[idxs]
    c_pars=c_pars[idxs]
    ## convert to tensors
    
    data_loader=f_sample_data(img,c_pars,gdict['world_rank'],gdict['world_size'])
    print("Size of dataset for GPU %s : %s"%(gdict['world_rank'],len(data_loader.dataset)))
    
    
    # Precompute metrics with validation data for computing losses
    with torch.no_grad():
        spec_mean_list=[];spec_sdev_list=[];hist_val_list=[]
        
        for count,sigma in enumerate(gdict['sigma_list']):
            ip_fname=gdict['ip_fname']+'/norm_1_sig_%s_train_val.npy'%(sigma)
            val_img=np.load(ip_fname,mmap_mode='r')[-10:].transpose(0,1,2,3,4).copy() ## Mod for 3D
            t_val_img=torch.from_numpy(val_img).to(gdict['device'])

            # Precompute radial coordinates
            if count==0: 
                r,ind=f_get_rad(val_img)
                r=r.to(gdict['device']); ind=ind.to(gdict['device'])
            # Stored mean and std of spectrum for full input data once
            mean_spec_val,sdev_spec_val=f_torch_image_spectrum(f_invtransform(t_val_img),1,r,ind)
            hist_val=f_compute_hist(t_val_img,bins=gdict['bns'])
            
            spec_mean_list.append(mean_spec_val)
            spec_sdev_list.append(sdev_spec_val)
            hist_val_list.append(hist_val)
        spec_mean_tnsr=torch.stack(spec_mean_list)
        spec_sdev_tnsr=torch.stack(spec_sdev_list)
        hist_val_tnsr=torch.stack(hist_val_list)
        
        del val_img; del t_val_img; del img; del spec_mean_list; del spec_sdev_list; del hist_val_list        

    return data_loader,spec_mean_tnsr,spec_sdev_tnsr,hist_val_tnsr,r,ind

In [24]:
# def f_sample_data(ip_arr,params_arr,rank=0,num_ranks=1):
#     '''
#     Module to load part of dataset depending on world_rank.
#     '''
#     data_size=ip_arr.shape[0]
#     size=data_size//num_ranks
    
#     ### Get a set of random indices from numpy array
#     idxs=np.arange(ip_arr.shape[0])
#     np.random.shuffle(idxs)
#     rnd_idxs=idxs[rank*(size):(rank+1)*size]
    
#     t_img=torch.from_numpy(ip_arr[rnd_idxs].copy())
#     cosm_params=torch.Tensor(params_arr[rnd_idxs]).view(size,1)
#     logging.info("%s, %s"%(cosm_params.shape,t_img.shape))

#     print(t_img.shape)
#     dataset=TensorDataset(t_img,cosm_params)
#     del t_img
#     ### 
#     if gdict['batch_size']>size:
#         print("Caution: batchsize %s is less than samples per GPU."%(gdict['batch_size'],size))
#         raise SystemExit
    
#     data_loader=DataLoader(dataset,batch_size=gdict['batch_size'],shuffle=True,num_workers=0,drop_last=True)
#     return data_loader


# def f_load_data_precompute(gdict):
#     #################################
#     ####### Read data and precompute ######
#     ### Read input data from different files
#     for count,sigma in enumerate(gdict['sigma_list']):
#         fname=gdict['ip_fname']+'/norm_1_sig_%s_train_val.npy'%(sigma)
#         x=np.load(fname,mmap_mode='r')[:gdict['num_imgs']].transpose(0,1,2,3,4) ## Mod for 3D
#         size=x.shape[0]
#         y=sigma*np.ones(size)
        
#         if count==0:
#             img=x[:]
#             c_pars=y[:]
#         else: 
#             img=np.vstack([img,x]) # Store images
#             c_pars=np.hstack([c_pars,y]) # Store cosmological parameters
    
#     ### Manually shuffling numpy arrays to mix sigma values
#     size=img.shape[0]
#     idxs=np.random.choice(size,size=size,replace=False)
#     img=img[idxs]
#     c_pars=c_pars[idxs]
#     ## convert to tensors
    
#     data_loader=f_sample_data(img,c_pars,gdict['world_rank'],gdict['world_size'])
#     print("Size of dataset for GPU %s : %s"%(gdict['world_rank'],len(data_loader.dataset)))
    
    
#     # Precompute metrics with validation data for computing losses
#     with torch.no_grad():
#         spec_mean_list=[];spec_sdev_list=[];hist_val_list=[]
        
#         for count,sigma in enumerate(gdict['sigma_list']):
#             ip_fname=gdict['ip_fname']+'/norm_1_sig_%s_train_val.npy'%(sigma)
#             val_img=np.load(ip_fname,mmap_mode='r')[-10:].transpose(0,1,2,3,4).copy() ## Mod for 3D
#             t_val_img=torch.from_numpy(val_img).to(gdict['device'])

#             # Precompute radial coordinates
#             if count==0: 
#                 r,ind=f_get_rad(val_img)
#                 r=r.to(gdict['device']); ind=ind.to(gdict['device'])
#             # Stored mean and std of spectrum for full input data once
#             mean_spec_val,sdev_spec_val=f_torch_image_spectrum(f_invtransform(t_val_img),1,r,ind)
#             hist_val=f_compute_hist(t_val_img,bins=gdict['bns'])
            
#             spec_mean_list.append(mean_spec_val)
#             spec_sdev_list.append(sdev_spec_val)
#             hist_val_list.append(hist_val)
#         spec_mean_tnsr=torch.stack(spec_mean_list)
#         spec_sdev_tnsr=torch.stack(spec_sdev_list)
#         hist_val_tnsr=torch.stack(hist_val_list)
        
#         del val_img; del t_val_img; del img; del spec_mean_list; del spec_sdev_list; del hist_val_list        

#     return data_loader,spec_mean_tnsr,spec_sdev_tnsr,hist_val_tnsr,r,ind

In [25]:
def f_get_img_samples(ip_arr,rank=0,num_ranks=1):
    '''
    Module to get part of the numpy image file
    '''
    
    data_size=ip_arr.shape[0]
    size=data_size//num_ranks
    
    if gdict['batch_size']>size:
        print("Caution: batchsize %s is greater than samples per GPU %s"%(gdict['batch_size'],size))
        raise SystemExit
        
    ### Get a set of random indices from numpy array
    idxs=np.arange(ip_arr.shape[0])
    np.random.shuffle(idxs)
    rnd_idxs=idxs[rank*(size):(rank+1)*size]
    
    arr=ip_arr[rnd_idxs].copy()
    return arr


def f_load_data_precompute(gdict):
    #################################
    ####### Read data and precompute ######
    ### Read input data from different files
    for count,sigma in enumerate(gdict['sigma_list']):
        fname=gdict['ip_fname']+'/norm_1_sig_%s_train_val.npy'%(sigma)
        x=np.load(fname,mmap_mode='r')[:gdict['num_imgs']].transpose(0,1,2,3,4) ## Mod for 3D
        x=f_get_img_samples(x,gdict['world_rank'],gdict['world_size'])
        size=x.shape[0]
        y=sigma*np.ones(size)
        
        if count==0:
            img=x[:]
            c_pars=y[:]
        else: 
            img=np.vstack([img,x]) # Store images
            c_pars=np.hstack([c_pars,y]) # Store cosmological parameters
    
    ### Manually shuffling numpy arrays to mix sigma values
    size=img.shape[0]
    idxs=np.random.choice(size,size=size,replace=False)
    img=img[idxs]
    c_pars=c_pars[idxs]
    ## convert to tensors
    t_img=torch.from_numpy(img)
    cosm_params=torch.Tensor(c_pars).view(size,1)

    dataset=TensorDataset(t_img,cosm_params)
    data_loader=DataLoader(dataset,batch_size=gdict['batch_size'],shuffle=True,num_workers=0,drop_last=True)
#     data_loader=f_sample_data(img,c_pars,gdict['world_rank'],gdict['world_size'])
    print("Size of dataset for GPU %s : %s"%(gdict['world_rank'],len(data_loader.dataset)))
    
    
    # Precompute metrics with validation data for computing losses
    with torch.no_grad():
        spec_mean_list=[];spec_sdev_list=[];hist_val_list=[]
        
        for count,sigma in enumerate(gdict['sigma_list']):
            ip_fname=gdict['ip_fname']+'/norm_1_sig_%s_train_val.npy'%(sigma)
            val_img=np.load(ip_fname,mmap_mode='r')[-10:].transpose(0,1,2,3,4).copy() ## Mod for 3D
            t_val_img=torch.from_numpy(val_img).to(gdict['device'])

            # Precompute radial coordinates
            if count==0: 
                r,ind=f_get_rad(val_img)
                r=r.to(gdict['device']); ind=ind.to(gdict['device'])
            # Stored mean and std of spectrum for full input data once
            mean_spec_val,sdev_spec_val=f_torch_image_spectrum(f_invtransform(t_val_img),1,r,ind)
            hist_val=f_compute_hist(t_val_img,bins=gdict['bns'])
            
            spec_mean_list.append(mean_spec_val)
            spec_sdev_list.append(sdev_spec_val)
            hist_val_list.append(hist_val)
        spec_mean_tnsr=torch.stack(spec_mean_list)
        spec_sdev_tnsr=torch.stack(spec_sdev_list)
        hist_val_tnsr=torch.stack(hist_val_list)
        
        del val_img; del t_val_img; del img; del spec_mean_list; del spec_sdev_list; del hist_val_list        

    return data_loader,spec_mean_tnsr,spec_sdev_tnsr,hist_val_tnsr,r,ind

## Main

In [26]:
#########################
### Main code #######
#########################

if __name__=="__main__":
    jpt=False
    jpt=True ##(different for jupyter notebook)
    t0=time.time()
    args=f_parse_args() if not jpt else f_manual_add_argparse()

    #################################
    ### Set up global dictionary###
    gdict={}
    gdict=f_init_gdict(args,gdict)
#     gdict['num_imgs']=200

    if jpt: ## override for jpt nbks
        gdict['num_imgs']=50
        gdict['run_suffix']='nb_test'
        
    f_setup(gdict,log=(not jpt))
    
    ## Build GAN
    netG,netD,criterion,optimizerD,optimizerG=f_init_GAN(gdict,print_model=True)
    fixed_noise = torch.randn(gdict['batch_size'], 1, 1, 1, gdict['nz'], device=gdict['device']) #Latent vectors to view G progress    # Mod for 3D
    rnd_idx=torch.randint(len(gdict['sigma_list']),(gdict['batch_size'],1),device=gdict['device'])
    fixed_cosm_params=torch.tensor([gdict['sigma_list'][i] for i in rnd_idx.long()],device=gdict['device']).unsqueeze(-1)
    
    if gdict['distributed']:  try_barrier(gdict['world_rank'])

    ## Load data and precompute
    dataloader,mean_spec_val,sdev_spec_val,hist_val,r,ind=f_load_data_precompute(gdict)

    #################################
    ########## Train loop and save metrics and images ######
    ### Set up metrics dataframe
    cols=['step','epoch','Dreal','Dfake','Dfull','G_adv','G_full','spec_loss','hist_loss','spec_chi','hist_chi','gp_loss','fm_loss','D(x)','D_G_z1','D_G_z2','time']
    metrics_df=pd.DataFrame(columns=cols)
    if gdict['distributed']:  try_barrier(gdict['world_rank'])
    
    logging.info("Starting Training Loop...")
    f_train_loop(dataloader,metrics_df,gdict,fixed_noise,mean_spec_val,sdev_spec_val,hist_val,r,ind)
    
    if gdict['world_rank']==0: ## Generate images for best saved models ######
        for cl in gdict['sigma_list']:
            op_loc=gdict['save_dir']+'/images/'
            ip_fname=gdict['save_dir']+'/models/checkpoint_best_spec.tar'
            f_gen_images(gdict,netG,optimizerG,cl,ip_fname,op_loc,op_strg='gen_img_best_spec',op_size=200)

            ip_fname=gdict['save_dir']+'/models/checkpoint_best_hist.tar'
            f_gen_images(gdict,netG,optimizerG,cl,ip_fname,op_loc,op_strg='gen_img_best_hist',op_size=200)

    
    tf=time.time()
    logging.info("Total time %s"%(tf-t0))
    logging.info('End: %s'%(datetime.now().strftime('%Y-%m-%d  %H:%M:%S')))

Not using DDP
Model name 2


RuntimeError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 15.78 GiB total capacity; 303.82 MiB already allocated; 10.75 MiB free; 362.00 MiB reserved in total by PyTorch)

In [12]:
%tb

SystemExit: 

In [ ]:
# metrics_df.plot('step','time')
# metrics_df.plot(x='step',y=['hist_loss','spec_chi'],kind='line')
metrics_df

In [ ]:
gdict

In [ ]:
pwd

In [ ]:
# for i in iter(dataloader):
#     print(i[0].shape)

### Debug

In [ ]:
# Feature matching loss


In [ ]:
# netG = Generator(gdict).to(gdict['device'])
# netG.apply(weights_init)
# # # #     print(netG)
# # summary(netG,(1,1,64))
# # Create Discriminator
# netD = Discriminator(gdict).to(gdict['device'])
# netD.apply(weights_init)
# #     print(netD)
# summary(netD,(1,128,128))

In [ ]:
# noise = torch.randn(gdict['batchsize'], 1, 1, gdict['nz'], device=gdict['device'])
# fake = netG(noise)            
# # Forward pass real batch through D
# output = netD(fake)
# print([i.shape for i in output])

In [ ]:
####### Read data and precompute ######
img=np.load(gdict['ip_fname'],mmap_mode='r')[:gdict['num_imgs']].transpose(0,1,2,3).copy()
t_img=torch.from_numpy(img)
print("%s, %s"%(img.shape,t_img.shape))

dataset=TensorDataset(t_img)
data_loader=DataLoader(dataset,batch_size=gdict['batch_size'],shuffle=True,num_workers=0,drop_last=True)

In [ ]:
len(data_loader.dataset)

In [ ]:
def f_sample_data(ip_tensor,rank=0,num_ranks=1):
    
    data_size=ip_tensor.shape[0]
    size=data_size//num_ranks
    print(size)
    print(rank*(size),(rank+1)*size)
    dataset=TensorDataset(ip_tensor[rank*(size):(rank+1)*size])
    data_loader=DataLoader(dataset,batch_size=gdict['batch_size'],shuffle=True,num_workers=0,drop_last=True)
    print(len(data_loader.dataset))
    return data_loader

f_sample_data(t_img,3,5)

In [ ]:
dir(dataloader)

In [ ]:
dataloader.dataset.tensors[0].shape

In [ ]:
for i in dataloader:
    print(i[0].shape)